# ARCHS4 model building with CLAMP

💡 **Environment:** `clamp-analyses`  

This notebook focuses on running CLAMPfull on ARCHS4 data to build and evaluate predictive models for gene regulation.

## Load libraries

In [18]:
if (!requireNamespace("CLAMP", quietly = TRUE)) {
    devtools::install_github("wgmao/CLAMP")
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(CLAMP)
library(CLAMP)

source(here("config.R"))

set.seed(config$ARCHS4$RANDOM_SVD_SEED)

## Output directory

In [19]:
output_dir <- config$ARCHS4$DATASET_FOLDER
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

In [20]:
archs4_svdRes <- readRDS(file.path(output_dir, "archs4_svdRes.rds"))
archs4_fbm_filt <- readRDS(file.path(output_dir, "archs4_fbm_filt.rds"))
archs4_baseRes <- readRDS(file.path(output_dir, "archs4_baseRes.rds"))
CLAMP_K_archs4 <- readRDS(file.path(output_dir, "CLAMP_K_archs4.rds"))
archs4_genes <- readRDS(file.path(output_dir, "archs4_genes.rds"))
sample_names <- readRDS(file.path(output_dir, "archs4_samples.rds"))

## Prepare pathway priors

In [ ]:
# # # run localy since server not have internet access

# archs4_gmtList <- list(
#   KEGG = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"),
#   BP = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
#   GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023")
# )

# # prefix each gene‐set name with its library to guarantee uniqueness
# for(lib in names(archs4_gmtList)) {
#   names(archs4_gmtList[[lib]]) <- paste0(lib, "_", names(archs4_gmtList[[lib]]))
# }

# archs4_pathMat <- gmtListToSparseMat(archs4_gmtList)
# saveRDS(archs4_pathMat, file = file.path(here('data/archs4'), "archs4_pathMat.rds"))

Auto-detected name: KEGG_2021_Human

Using cached file for KEGG_2021_Human

Auto-detected name: GO_Biological_Process_2025

Using cached file for GO_Biological_Process_2025

Auto-detected name: GTEx_Tissues_V8_2023

Using cached file for GTEx_Tissues_V8_2023



In [ ]:
archs4_pathMat <- readRDS(file.path(here('data/archs4'), "archs4_pathMat.rds"))
archs4_matched <- getMatchedPathwayMat(archs4_pathMat, archs4_genes)
archs4_chatObj <- getChat(archs4_matched)

There are 12617 genes in the intersection between data and prior

Removing 2204 pathways

Inverting...

done



In [23]:
saveRDS(archs4_pathMat, file = file.path(output_dir, "archs4_pathMat.rds"))
saveRDS(archs4_matched, file = file.path(output_dir, "archs4_matched.rds"))
saveRDS(archs4_chatObj, file = file.path(output_dir, "archs4_chatObj.rds"))

## CLAMPfull

In [24]:
archs4_fullRes <- CLAMPfull(
    Y = archs4_fbm_filt,
    svdres = archs4_svdRes,
    priorMat = archs4_matched,
    clamp.base.result = archs4_baseRes,
    use_cpp = TRUE,
    trace = TRUE,
    clamp_k = CLAMP_K_archs4
  )

** CLAMPfull **

using provided CLAMPbase result

CLAMP k is set to 162

L1=19.2919187521306; L2=57.8757562563918

Progress 1 / 30 | Bdiff=0.000431

Progress 2 / 30 | Bdiff=0.011936

Progress 3 / 30 | Bdiff=0.006105

Estimated total runtime: ~2.1 min

Progress 4 / 30 | Bdiff=0.004897

Progress 5 / 30 | Bdiff=0.005179

Progress 6 / 30 | Bdiff=0.007165

Progress 7 / 30 | Bdiff=0.004079

Progress 8 / 30 | Bdiff=0.003886

Progress 9 / 30 | Bdiff=0.005650

Progress 10 / 30 | Bdiff=0.005829

Progress 11 / 30 | Bdiff=0.004823

Progress 12 / 30 | Bdiff=0.003509

Progress 13 / 30 | Bdiff=0.003234

Progress 14 / 30 | Bdiff=0.002730

Progress 15 / 30 | Bdiff=0.002792

Progress 16 / 30 | Bdiff=0.002272

Progress 17 / 30 | Bdiff=0.001959

Progress 18 / 30 | Bdiff=0.001741

Progress 19 / 30 | Bdiff=0.001521

Progress 20 / 30 | Bdiff=0.001656

Progress 21 / 30 | Bdiff=0.001998

Progress 22 / 30 | Bdiff=0.001753

Progress 23 / 30 | Bdiff=0.001629

Progress 24 / 30 | Bdiff=0.001892

Progress 25 / 30 | 

In [25]:
archs4_fullRes$Z <- data.frame(archs4_baseRes$Z)
rownames(archs4_baseRes$Z) <- archs4_genes
head(archs4_baseRes$Z)

archs4_fullRes$B <- data.frame(archs4_fullRes$B)
colnames(archs4_fullRes$B) <- sample_names
head(archs4_fullRes$B)

archs4_fullRes$summary <- archs4_fullRes$summary %>%
    dplyr::rename(LV = LV_index)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

head(archs4_fullRes$summary)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV153,LV154,LV155,LV156,LV157,LV158,LV159,LV160,LV161,LV162
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,0,0,0,0.0000000,0,0.1447130,0,0,0,0.1686867,⋯,0.00000000,0,0.0000000,0.0000000,0,0.0000000,0.000000,0,0,0.0000000
A1BG-AS1,0,0,0,0.0000000,0,0.2758320,0,0,0,0.0000000,⋯,0.00000000,0,0.0000000,0.2450749,0,0.0000000,0.000000,0,0,0.0000000
A2M,0,0,0,0.2633904,0,0.0000000,0,0,0,0.0000000,⋯,0.22824428,0,0.0000000,0.3939044,0,0.2862303,0.221922,0,0,0.2620452
A4GALT,0,0,0,0.3207190,0,0.3502162,0,0,0,0.0000000,⋯,0.03349063,0,0.0000000,0.0000000,0,0.0000000,0.000000,0,0,0.0000000
AAAS,0,0,0,0.0000000,0,0.2541471,0,0,0,0.0000000,⋯,0.00000000,0,0.0000000,0.0000000,0,0.0000000,0.000000,0,0,0.0000000
AACS,0,0,0,0.0000000,0,0.2842216,0,0,0,0.0000000,⋯,0.00000000,0,0.1614475,0.0000000,0,0.0000000,0.000000,0,0,0.0000000


,GSM4372617,GSM5491449,GSM3127398,GSM5957474,GSM3573906,GSM5633169,GSM4372072,GSM5358182,GSM4660685,GSM6656371,⋯,GSM6244357,GSM5101191,GSM2800621,GSM3612465,GSM8359157,GSM4649264,GSM5402786,GSM5174325,GSM5344761,GSM7763940
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LV1,0.55265094,0.53606165,0.61488811,0.673309627,0.63516825,-0.79105825,-0.749720258,-0.7315398674,-0.81165444,-0.791243642,⋯,1.16287428,1.18654902,1.0520342,1.0905804,1.20692201,1.13834810,0.98052859,1.01143903,1.14687888,1.0812003
LV2,-0.22695731,-0.23647944,-0.23343655,-0.280578343,-0.28152976,0.12033924,0.157023720,0.1203278014,0.13357047,0.147427686,⋯,-0.23161118,-0.27187806,-0.2435178,-0.2732431,-0.21088709,-0.25520319,-0.24428834,-0.28121515,-0.23966873,-0.2244001
LV3,-0.43042000,-0.39466906,-0.41487177,-0.430353105,-0.39503102,-0.04927134,-0.080435858,-0.0004841786,-0.07648530,-0.046562837,⋯,-0.56067038,-0.61997564,-0.6714663,-0.5973300,-0.65969400,-0.64761787,-0.67527750,-0.60292641,-0.59954208,-0.6097347
LV4,-0.01913349,-0.02137825,-0.01390390,0.002168199,-0.00116251,-0.11734224,-0.091458769,-0.1304812642,-0.09375273,-0.068830862,⋯,-0.10003740,-0.14781652,-0.1016215,-0.1334954,-0.09764781,-0.09707139,-0.13770381,-0.09442266,-0.12690568,-0.1042092
LV5,-0.08233468,-0.08779257,-0.08127942,-0.100600831,-0.09183778,-0.02378350,-0.007456219,-0.0291567271,-0.01225229,-0.006625202,⋯,-0.17360481,-0.04627084,-0.1289069,-0.0853934,-0.16256369,-0.03708651,-0.03566701,-0.21658730,-0.04998241,-0.1107387
LV6,0.82400018,0.86935563,0.72456528,0.798331398,0.70986346,-1.00300651,-0.999417378,-1.0170561402,-0.95978746,-0.993808147,⋯,-0.07667684,0.70000480,0.7189658,0.8982753,0.31188372,1.06668630,1.14066177,1.08548982,1.18768279,1.1563377


,pathway,LV,AUC,p_value,FDR,npos,nneg
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>
BP532,BP_Camera-Type Eye Development (GO:0043010),LV1,0.4694626,0.5847960,0.7449913,4,16488
BP630,BP_Cellular Response to L-leucine (GO:0071233),LV1,0.4839580,0.5317297,0.7006320,2,16488
BP1398,BP_Insulin-Like Growth Factor Receptor Signaling Pathway (GO:0048009),LV1,0.6092310,0.2940673,0.5073962,2,16488
BP3249,BP_Positive Regulation of Muscle Hypertrophy (GO:0014742),LV1,0.4490943,0.6214137,0.7634187,3,16488
BP3577,BP_Protein K48-linked Deubiquitination (GO:0071108),LV1,0.6596717,0.1661292,0.3419260,3,16488
BP3593,BP_Protein auto-ADP-ribosylation (GO:0070213),LV1,0.5622574,0.3788033,0.5772241,2,16488


In [26]:
saveRDS(archs4_fullRes, file = file.path(output_dir, "CLAMPfull.rds"))

In [27]:
model_dir <- file.path(output_dir, "CLAMPfull")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- archs4_fullRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- archs4_fullRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- archs4_fullRes$summary
write.csv(summary, file.path(model_dir, "summary.csv"))

In [ ]:
saveRDS(archs4_fullRes, file = file.path(output_dir, "archs4_CLAMP.rds"))